In [2]:
import os
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from PIL import Image

# Intel Image Dataset Classification Section

In [3]:
data_dir= '/kaggle/input/intel-image-classification'
transform=transforms.Compose([
transforms.Resize((224,224)),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485,0.456,0.406],
                    std=[0.229,0.224,0.225])
])
train_dataset=ImageFolder(root=os.path.join(data_dir,'seg_train'),transform =transform)
test_dataset=ImageFolder(root=os.path.join(data_dir,'seg_test'),transform =transform)

train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32)

In [4]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=resnet18(pretrained=True)
for param in model.parameters():
    param.required_grad=False

model.fc=nn.Linear(model.fc.in_features,2)
model=model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 204MB/s]


In [5]:
criterion =nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.fc.parameters(),lr=1e-3)

In [1]:
import torch
print("✅ CUDA available:", torch.cuda.is_available())
print("🖥️ Current device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


✅ CUDA available: True
🖥️ Current device: Tesla P100-PCIE-16GB


In [6]:
def train_model(model, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            if (batch_idx + 1) % 10 == 0 or (batch_idx + 1) == len(train_loader):
                print(f"  Batch {batch_idx+1}/{len(train_loader)} | Loss: {loss.item():.4f}")

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {100 * correct / total:.2f}%")
train_model(model, epochs=5)

  Batch 10/439 | Loss: 0.0057
  Batch 20/439 | Loss: 0.0006
  Batch 30/439 | Loss: 0.0003
  Batch 40/439 | Loss: 0.0003
  Batch 50/439 | Loss: 0.0002
  Batch 60/439 | Loss: 0.0004
  Batch 70/439 | Loss: 0.0002
  Batch 80/439 | Loss: 0.0002
  Batch 90/439 | Loss: 0.0002
  Batch 100/439 | Loss: 0.0004
  Batch 110/439 | Loss: 0.0003
  Batch 120/439 | Loss: 0.0002
  Batch 130/439 | Loss: 0.0002
  Batch 140/439 | Loss: 0.0001
  Batch 150/439 | Loss: 0.0002
  Batch 160/439 | Loss: 0.0002
  Batch 170/439 | Loss: 0.0002
  Batch 180/439 | Loss: 0.0001
  Batch 190/439 | Loss: 0.0001
  Batch 200/439 | Loss: 0.0007
  Batch 210/439 | Loss: 0.0001
  Batch 220/439 | Loss: 0.0001
  Batch 230/439 | Loss: 0.0001
  Batch 240/439 | Loss: 0.0001
  Batch 250/439 | Loss: 0.0001
  Batch 260/439 | Loss: 0.0001
  Batch 270/439 | Loss: 0.0001
  Batch 280/439 | Loss: 0.0002
  Batch 290/439 | Loss: 0.0001
  Batch 300/439 | Loss: 0.0001
  Batch 310/439 | Loss: 0.0001
  Batch 320/439 | Loss: 0.0002
  Batch 330/439 |

In [7]:
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

evaluate(model)


Test Accuracy: 100.00%


In [9]:
model_save_path = "/kaggle/working/resnet_disaster_clean.pt"
torch.save(model.state_dict(), model_save_path)

In [10]:
import os
print(os.path.exists("/kaggle/working/resnet_disaster_clean.pt"))  # Should print True


True


# Xview2 Model

In [2]:
import os
import json
import pandas as pd

# Path to the folder containing JSON files
json_dir = "/kaggle/input/labels/labels"

data = []

# Loop through all JSON files
for filename in os.listdir(json_dir):
    if filename.endswith(".json"):
        filepath = os.path.join(json_dir, filename)

        with open(filepath, 'r') as f:
            try:
                label_data = json.load(f)
                
                # You can customize this extraction as needed
                image_id = filename.replace('_post_disaster.json', '')
                features = label_data.get("features", {}).get("xy", [])
                label_type = label_data.get("metadata", {}).get("disaster_type", "unknown")

                data.append({
                    "image_id": image_id,
                    "num_features": len(features),
                    "disaster_type": label_type
                })
            except Exception as e:
                print(f"Error processing {filename}: {e}")

# Create a DataFrame and save as CSV
df = pd.DataFrame(data)
df.to_csv("/kaggle/working/train_labels.csv", index=False)

print("✅ CSV saved to /kaggle/working/train_labels.csv")


✅ CSV saved to /kaggle/working/train_labels.csv


In [1]:
!pip install -q gdown

In [14]:
import os, tarfile, time, torch
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from glob import glob
from tqdm import tqdm

# ========== CONFIG ==========
tgz_path = "/kaggle/input/xview2-geotiff-full/xview2_geotiff.tgz"
checkpoint_in_path = "/kaggle/input/resnet-disaster-clean/resnet_disaster_clean.pt"
checkpoint_out_path = "/kaggle/working/resnet_disaster_clean.pt"
labels_path = "/kaggle/input/train-labels/train_labels.csv"
extract_base = "/kaggle/temp_extract"
chunk_size = 50
num_classes = 4
batch_size = 8
resume_from_chunk = 38
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== LABELS ==========
labels_df = pd.read_csv(labels_path)
labels_df['image_id'] = labels_df['image_id'].astype(str) + '_post_disaster.tif'
label_dict = dict(zip(labels_df['image_id'], labels_df['disaster_type']))

# ========== TRANSFORM ==========
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ========== DATASET ==========
class SatelliteDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img_name = os.path.basename(img_path)

        import tifffile
        import numpy as np

        try:
        # Try reading image with tifffile
            image = tifffile.imread(img_path)
        # Convert grayscale to RGB or trim to 3 channels
            image = np.stack([image]*3, axis=-1) if image.ndim == 2 else image[..., :3]
            image = Image.fromarray(image.astype(np.uint8))
        except Exception as e:
            print(f"[!] Failed to read image: {img_path} -> {e}")
        # Skip to the next image in a circular way
            return self.__getitem__((idx + 1) % len(self.image_paths))

    # Apply transforms if any
        if self.transform:
            image = self.transform(image)

    # Get label from dict (fallback to 0)
        label = self.labels.get(img_name, 0)

        return image, label


# ========== TRAINING ==========
def train_one_epoch(model, dataloader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in tqdm(dataloader, desc="🔁 Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total

# ========== TGZ CHUNK PROCESSING ==========
import time

def process_tgz_in_chunks(tgz_path, extract_base, chunk_size=50):
    print(f"📦 Opening archive: {tgz_path}")
    start = time.time()

    with tarfile.open(tgz_path, "r:gz") as tar:
        print("🔍 Scanning archive for post-disaster images...")
        post_members = []
        total = 0
        for m in tar:
            total += 1
            if total % 1000 == 0:
                print(f"  └─ Scanned {total} files...")
            if "_post_disaster.tif" in m.name and m.isfile():
                post_members.append(m)

        print(f"✅ Found {len(post_members)} post-disaster images in {time.time() - start:.2f} seconds.")

        for i in range(0, len(post_members), chunk_size):
            chunk_members = post_members[i:i + chunk_size]
            chunk_dir = os.path.join(extract_base, f"chunk_{i//chunk_size}")
            os.makedirs(chunk_dir, exist_ok=True)

            print(f"\n📂 Extracting chunk {i//chunk_size + 1} with {len(chunk_members)} images to {chunk_dir}...")
            for member in chunk_members:
                member.name = os.path.basename(member.name)  # Flatten to avoid nested dirs
                tar.extract(member, path=chunk_dir)

            yield chunk_dir

            # Delete after use to free space
            print(f"🧹 Cleaning up extracted files in {chunk_dir}")
            shutil.rmtree(chunk_dir)

# ========== MODEL ==========
def create_model(num_classes, checkpoint_path=None):
    model = models.resnet18(weights="IMAGENET1K_V1")
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        filtered_checkpoint = {k: v for k, v in checkpoint.items() if k in model.state_dict() and v.size() == model.state_dict()[k].size()}
        model.load_state_dict(filtered_checkpoint, strict=False)
        print(f"✅ Loaded partial weights from: {checkpoint_path}")
        ignored = [k for k in checkpoint if k not in filtered_checkpoint]
        if ignored:
            print(f"ℹ️ Ignored layers: {ignored}")
    return model

# ========== MAIN ==========
model = create_model(num_classes, checkpoint_in_path)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for chunk_dir in process_tgz_in_chunks(tgz_path, extract_base, chunk_size):
    image_paths = sorted(glob(os.path.join(chunk_dir, "*.tif")))
    if not image_paths:
        print(f"🚫 No images found in {chunk_dir}. Skipping...")
        continue

    dataset = SatelliteDataset(image_paths, label_dict, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    loss, acc = train_one_epoch(model, dataloader, criterion, optimizer)
    print(f"📊 Chunk Loss = {loss:.4f}, Accuracy = {acc:.4f}")

# ========== SAVE ==========
torch.save(model.state_dict(), checkpoint_out_path)
print(f"💾 Final model saved to {checkpoint_out_path}")


✅ Loaded partial weights from: /kaggle/input/resnet-disaster-clean/resnet_disaster_clean.pt
ℹ️ Ignored layers: ['fc.weight', 'fc.bias']
📦 Opening archive: /kaggle/input/xview2-geotiff-full/xview2_geotiff.tgz
🔍 Scanning archive for post-disaster images...
  └─ Scanned 1000 files...
  └─ Scanned 2000 files...
  └─ Scanned 3000 files...
  └─ Scanned 4000 files...
  └─ Scanned 5000 files...
  └─ Scanned 6000 files...
  └─ Scanned 7000 files...
  └─ Scanned 8000 files...
  └─ Scanned 9000 files...
  └─ Scanned 10000 files...
  └─ Scanned 11000 files...
  └─ Scanned 12000 files...
  └─ Scanned 13000 files...
  └─ Scanned 14000 files...
  └─ Scanned 15000 files...
  └─ Scanned 16000 files...
  └─ Scanned 17000 files...
  └─ Scanned 18000 files...
  └─ Scanned 19000 files...
  └─ Scanned 20000 files...
  └─ Scanned 21000 files...
  └─ Scanned 22000 files...
  └─ Scanned 23000 files...
  └─ Scanned 24000 files...
  └─ Scanned 25000 files...
  └─ Scanned 26000 files...
  └─ Scanned 27000 files..

📊 Chunk Loss = 0.8898, Accuracy = 0.7800
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_0

📂 Extracting chunk 2 with 50 images to /kaggle/temp_extract/chunk_1...


📊 Chunk Loss = 0.5608, Accuracy = 0.9800
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_1

📂 Extracting chunk 3 with 50 images to /kaggle/temp_extract/chunk_2...


📊 Chunk Loss = 0.3420, Accuracy = 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_2

📂 Extracting chunk 4 with 50 images to /kaggle/temp_extract/chunk_3...


📊 Chunk Loss = 0.2156, Accuracy = 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_3

📂 Extracting chunk 5 with 50 images to /kaggle/temp_extract/chunk_4...


...

📊 Chunk Loss = 0.0072, Accuracy = 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_33

📂 Extracting chunk 35 with 50 images to /kaggle/temp_extract/chunk_34...


📊 Chunk Loss = 0.0068, Accuracy = 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_34

📂 Extracting chunk 36 with 50 images to /kaggle/temp_extract/chunk_35...


📊 Chunk Loss = 0.0065, Accuracy = 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_35

📂 Extracting chunk 37 with 50 images to /kaggle/temp_extract/chunk_36...


📊 Chunk Loss = 0.0064, Accuracy = 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_36

📂 Extracting chunk 38 with 50 images to /kaggle/temp_extract/chunk_37...


AttributeError: 'tuple' object has no attribute 'to'

In [1]:
import os, tarfile, time, shutil
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
from glob import glob
from tqdm import tqdm
import torch

# ========== CONFIG ==========
tgz_path = "/kaggle/input/xview2-geotiff-full/xview2_geotiff.tgz"
checkpoint_in_path = "/kaggle/input/resnet-disaster-clean/resnet_disaster_clean.pt"
checkpoint_out_path = "/kaggle/working/resnet_disaster_clean.pt"
labels_path = "/kaggle/input/train-labels/train_labels.csv"
extract_base = "/kaggle/temp_extract"
chunk_size = 50
num_classes = 4
batch_size = 8
resume_from_chunk = 38
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== LABELS ==========
label_map = {
    "earthquake": 0,
    "flood": 1,
    "fire": 2,
    "tsunami": 3
}

labels_df = pd.read_csv(labels_path)
labels_df['image_id'] = labels_df['image_id'].astype(str) + '_post_disaster.tif'
labels_df['label_id'] = labels_df['disaster_type'].map(label_map)
label_dict = dict(zip(labels_df['image_id'], labels_df['label_id']))

# ========== TRANSFORM ==========
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ========== SAFE COLLATE ==========
def safe_collate(batch):
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        return None
    images, labels = zip(*batch)
    images = torch.stack(images)
    labels = torch.tensor(labels)
    return images, labels

# ========== DATASET ==========
class SatelliteDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img_name = os.path.basename(img_path)

        import tifffile
        import numpy as np
        from PIL import Image
        import torch

        for _ in range(3):
            try:
                image = tifffile.imread(img_path)
                if image.ndim == 2:
                    image = np.stack([image] * 3, axis=-1)
                else:
                    image = image[..., :3]

                image = Image.fromarray(image.astype(np.uint8))
                if self.transform:
                    image = self.transform(image)

                label = self.labels.get(img_name, 0)
                label = int(label)  # 🔧 Ensure label is int
                return image, label

            except Exception as e:
                print(f"[!] Skipping unreadable image: {img_path} -> {e}")
                idx = (idx + 1) % len(self.image_paths)
                img_path = self.image_paths[idx]
                img_name = os.path.basename(img_path)

        dummy = torch.zeros(3, 224, 224)
        return dummy, 0

# ========== TRAINING ==========
def train_one_epoch(model, dataloader, criterion, optimizer):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for batch in tqdm(dataloader, desc="🔁 Training", leave=False):
        if batch is None or not isinstance(batch, (tuple, list)) or len(batch) != 2:
            print(f"⚠️ Invalid batch structure: {type(batch)} → {batch}")
            continue

        images, labels = batch

        if isinstance(images, list):
            images = torch.stack(images)
        if isinstance(labels, list):
            labels = torch.tensor(labels)

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total if total > 0 else 0.0
    epoch_acc = correct / total if total > 0 else 0.0
    return epoch_loss, epoch_acc

# ========== TGZ CHUNK PROCESSING ==========
def process_tgz_in_chunks(tgz_path, extract_base, chunk_size=50, resume_from_chunk=0):
    print(f"📦 Opening archive: {tgz_path}")
    with tarfile.open(tgz_path, "r:gz") as tar:
        print("🔍 Scanning archive for post-disaster images...")
        post_members = []
        count = 0
        for member in tar:
            if "_post_disaster.tif" in member.name and member.isfile():
                post_members.append(member)
                count += 1
                if count % 1000 == 0:
                    print(f"  → Found {count} post-disaster images...")

        for i in range(0, len(post_members), chunk_size):
            chunk_index = i // chunk_size
            if chunk_index < resume_from_chunk:
                continue

            chunk_members = post_members[i:i + chunk_size]
            chunk_dir = os.path.join(extract_base, f"chunk_{chunk_index}")
            os.makedirs(chunk_dir, exist_ok=True)

            print(f"\n📂 Extracting chunk {chunk_index + 1} with {len(chunk_members)} images to {chunk_dir}...")
            for member in chunk_members:
                member.name = os.path.basename(member.name)
                tar.extract(member, path=chunk_dir)

            yield chunk_dir, chunk_index

            print(f"🧹 Cleaning up extracted files in {chunk_dir}")
            shutil.rmtree(chunk_dir)

# ========== MODEL ==========
def create_model(num_classes, checkpoint_path=None):
    model = models.resnet18(weights="IMAGENET1K_V1")
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        filtered_checkpoint = {k: v for k, v in checkpoint.items()
                               if k in model.state_dict() and v.size() == model.state_dict()[k].size()}
        model.load_state_dict(filtered_checkpoint, strict=False)
        print(f"✅ Loaded partial weights from: {checkpoint_path}")
        ignored = [k for k in checkpoint if k not in filtered_checkpoint]
        if ignored:
            print(f"ℹ️ Ignored layers: {ignored}")
    return model

# ========== MAIN ==========
model = create_model(num_classes, checkpoint_in_path)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for chunk_dir, chunk_index in process_tgz_in_chunks(tgz_path, extract_base, chunk_size, resume_from_chunk):
    try:
        image_paths = sorted(glob(os.path.join(chunk_dir, "*.tif")))
        if not image_paths:
            print(f"🚫 No images found in {chunk_dir}. Skipping...")
            continue

        dataset = SatelliteDataset(image_paths, label_dict, transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=safe_collate)

        if dataloader is None or len(dataloader) == 0:
            print(f"🚫 No valid images found in chunk {chunk_index + 1}. Skipping...")
            continue

        loss, acc = train_one_epoch(model, dataloader, criterion, optimizer)
        print(f"📊 Chunk {chunk_index + 1} → Loss: {loss:.4f}, Accuracy: {acc:.4f}")

        torch.save(model.state_dict(), checkpoint_out_path)

    except Exception as e:
        print(f"❌ Failed during training chunk {chunk_index + 1}: {e}")

# ========== SAVE ==========
torch.save(model.state_dict(), checkpoint_out_path)
print(f"💾 Final model saved to {checkpoint_out_path}")


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 163MB/s] 


✅ Loaded partial weights from: /kaggle/input/resnet-disaster-clean/resnet_disaster_clean.pt
ℹ️ Ignored layers: ['fc.weight', 'fc.bias']
📦 Opening archive: /kaggle/input/xview2-geotiff-full/xview2_geotiff.tgz
🔍 Scanning archive for post-disaster images...
  → Found 1000 post-disaster images...
  → Found 2000 post-disaster images...
  → Found 3000 post-disaster images...
  → Found 4000 post-disaster images...
  → Found 5000 post-disaster images...
  → Found 6000 post-disaster images...
  → Found 7000 post-disaster images...
  → Found 8000 post-disaster images...
  → Found 9000 post-disaster images...
  → Found 10000 post-disaster images...
  → Found 11000 post-disaster images...

📂 Extracting chunk 39 with 50 images to /kaggle/temp_extract/chunk_38...


🔁 Training:   0%|          | 0/7 [00:00<?, ?it/s]

[!] Skipping unreadable image: /kaggle/temp_extract/chunk_38/hurricane-florence_00000037_post_disaster.tif -> cannot convert float NaN to integer
[!] Skipping unreadable image: /kaggle/temp_extract/chunk_38/hurricane-florence_00000042_post_disaster.tif -> cannot convert float NaN to integer
[!] Skipping unreadable image: /kaggle/temp_extract/chunk_38/hurricane-florence_00000043_post_disaster.tif -> cannot convert float NaN to integer
[!] Skipping unreadable image: /kaggle/temp_extract/chunk_38/hurricane-florence_00000051_post_disaster.tif -> cannot convert float NaN to integer
[!] Skipping unreadable image: /kaggle/temp_extract/chunk_38/hurricane-florence_00000053_post_disaster.tif -> cannot convert float NaN to integer
[!] Skipping unreadable image: /kaggle/temp_extract/chunk_38/hurricane-florence_00000054_post_disaster.tif -> cannot convert float NaN to integer
[!] Skipping unreadable image: /kaggle/temp_extract/chunk_38/hurricane-florence_00000049_post_disaster.tif -> cannot convert

🔁 Training:  14%|█▍        | 1/7 [00:01<00:11,  1.88s/it]

...

📊 Chunk 217 → Loss: 0.0012, Accuracy: 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_216

📂 Extracting chunk 218 with 50 images to /kaggle/temp_extract/chunk_217...


📊 Chunk 218 → Loss: 0.0012, Accuracy: 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_217

📂 Extracting chunk 219 with 50 images to /kaggle/temp_extract/chunk_218...


📊 Chunk 219 → Loss: 0.0012, Accuracy: 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_218

📂 Extracting chunk 220 with 50 images to /kaggle/temp_extract/chunk_219...


📊 Chunk 220 → Loss: 0.0012, Accuracy: 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_219

📂 Extracting chunk 221 with 34 images to /kaggle/temp_extract/chunk_220...


📊 Chunk 221 → Loss: 0.0014, Accuracy: 1.0000
🧹 Cleaning up extracted files in /kaggle/temp_extract/chunk_220
💾 Final model saved to /kaggle/working/resnet_disaster_clean.pt


In [14]:
import os, tarfile, shutil
import pandas as pd
import numpy as np
import torch
import tifffile
from PIL import Image
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from glob import glob
from sklearn.metrics import classification_report
from tqdm import tqdm

# ========== CONFIG ==========
tgz_path = "/kaggle/input/xview2-geotiff-full/xview2_geotiff.tgz"
checkpoint_path = "/kaggle/input/resnet-disaster-clean/resnet_disaster_clean.pt"
labels_path = "/kaggle/input/train-labels/train_labels.csv"
extract_base = "/kaggle/temp_eval_extract"
target_samples = 150  # Total images to evaluate
batch_size = 8
num_classes = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== LABELS ==========
label_map = {"earthquake": 0, "flood": 1, "fire": 2, "tsunami": 3}
labels_df = pd.read_csv(labels_path)
labels_df["image_id"] = labels_df["image_id"].astype(str) + "_post_disaster.tif"
labels_df["label_id"] = labels_df["disaster_type"].map(label_map)
labels_df = labels_df.dropna(subset=['image_id', 'label_id'])
label_dict = dict(zip(labels_df['image_id'], labels_df['label_id']))

# ========== TRANSFORM ==========
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ========== SAFE COLLATE ==========
def safe_collate(batch):
    batch = [item for item in batch if item is not None]
    if not batch:
        return None
    images, labels = zip(*batch)
    return torch.stack(images), torch.tensor(labels)

# ========== DATASET ==========
class SatelliteDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img_name = os.path.basename(img_path)

        for _ in range(3):
            try:
                image = tifffile.imread(img_path)
                if image.ndim == 2:
                    image = np.stack([image] * 3, axis=-1)
                else:
                    image = image[..., :3]

                image = Image.fromarray(image.astype(np.uint8))
                if self.transform:
                    image = self.transform(image)

                label = int(self.labels.get(img_name, 0))
                return image, label
            except Exception as e:
                print(f"[!] Skipping unreadable image: {img_path} -> {e}")
                idx = (idx + 1) % len(self.image_paths)
                img_path = self.image_paths[idx]
                img_name = os.path.basename(img_path)

        dummy = torch.zeros(3, 224, 224)
        return dummy, 0

# ========== MODEL ==========
def load_model(checkpoint_path, num_classes):
    model = models.resnet18(weights="IMAGENET1K_V1")
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    model = model.to(device)

    checkpoint = torch.load(checkpoint_path, map_location=device)
    model_state = model.state_dict()

    filtered = {
        k: v for k, v in checkpoint.items()
        if k in model_state and v.size() == model_state[k].size()
    }

    ignored = [k for k in checkpoint if k not in filtered or checkpoint[k].size() != model_state.get(k, torch.empty(0)).size()]
    
    model.load_state_dict(filtered, strict=False)

    print(f"✅ Loaded checkpoint with partial matching layers.")
    if ignored:
        print(f"⚠️ Ignored layers due to size mismatch: {ignored}")

    return model.eval()

# ========== EXTRACT BALANCED EVAL IMAGES ==========
def extract_balanced_images(tgz_path, extract_base, label_dict, target_count=150):
    from collections import defaultdict

    print(f"📦 Streaming archive: {tgz_path}")
    tar = tarfile.open(tgz_path, "r:gz")

    label_counts = defaultdict(int)
    max_per_class = target_count // len(label_map)
    selected_members = []

    print(f"🔍 Scanning archive lazily for balanced classes...")

    scanned = 0
    for member in tar:
        scanned += 1
        if not member.isfile():
            continue
        if "_post_disaster.tif" not in member.name:
            continue

        img_name = os.path.basename(member.name)
        label = label_dict.get(img_name, None)

        if label is None or pd.isna(label):
            continue
        if label_counts[label] >= max_per_class:
            continue

        label_counts[label] += 1
        selected_members.append(member)

        if sum(label_counts.values()) >= target_count:
            break

        if scanned % 1000 == 0:
            print(f"📊 Scanned {scanned} files... Selected: {len(selected_members)}")

    if not selected_members:
        raise RuntimeError("❌ No balanced images could be extracted. Check label mapping.")

    extract_dir = os.path.join(extract_base, "eval_chunk")
    os.makedirs(extract_dir, exist_ok=True)

    print(f"📂 Extracting {len(selected_members)} images to {extract_dir}...")

    for member in selected_members:
        member.name = os.path.basename(member.name)  # Flatten structure
        try:
            tar.extract(member, path=extract_dir)
        except Exception as e:
            print(f"[!] Failed to extract {member.name}: {e}")

    tar.close()
    return sorted(glob(os.path.join(extract_dir, "*.tif")))

# ========== EVALUATION ==========
def evaluate(model, dataloader):
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="🧪 Evaluating", leave=False):
            if batch is None:
                continue
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_preds, all_labels

# ========== MAIN ==========
model = load_model(checkpoint_path, num_classes)
image_paths = extract_balanced_images(tgz_path, extract_base, label_dict, target_samples)
dataset = SatelliteDataset(image_paths, label_dict, transform)
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=safe_collate)
preds, labels = evaluate(model, dataloader)

print("\n📊 Classification Report:")
print(classification_report(labels, preds, target_names=label_map.keys()))

# ========== CLEANUP ==========
shutil.rmtree(extract_base)


✅ Loaded checkpoint with partial matching layers.
⚠️ Ignored layers due to size mismatch: ['fc.weight', 'fc.bias']
📦 Streaming archive: /kaggle/input/xview2-geotiff-full/xview2_geotiff.tgz
🔍 Scanning archive lazily for balanced classes...
📊 Scanned 10000 files... Selected: 27
📊 Scanned 11000 files... Selected: 88
📂 Extracting 111 images to /kaggle/temp_eval_extract/eval_chunk...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.


📊 Classification Report:
              precision    recall  f1-score   support

  earthquake       0.43      0.08      0.14        37
       flood       0.00      0.00      0.00         0
        fire       0.00      0.00      0.00        37
     tsunami       0.32      0.76      0.45        37

    accuracy                           0.28       111
   macro avg       0.19      0.21      0.15       111
weighted avg       0.25      0.28      0.20       111



In [5]:
print("Unique true labels:", np.unique(all_labels, return_counts=True))
print("Unique predicted labels:", np.unique(all_preds, return_counts=True))


Unique true labels: (array([0]), array([100]))
Unique predicted labels: (array([0]), array([100]))


In [15]:
from sklearn.metrics import classification_report

# Generate report string
report = classification_report(labels, preds, target_names=label_map.keys())

# Save to file
report_path = "classification_report.txt"
with open(report_path, "w") as f:
    f.write(report)

print(f"\n✅ Report saved as: {report_path}")



✅ Report saved as: classification_report.txt


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [16]:
import base64
from IPython.display import HTML

def create_download_link(filename):
    with open(filename, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()
    href = f'<a download="{filename}" href="data:text/plain;base64,{encoded}" target="_blank">📥 Click here to download {filename}</a>'
    return HTML(href)

create_download_link("classification_report.txt")
